# BANKs

In [66]:
import numpy as np
import pandas as pd

In [67]:
df = pd.read_csv('bank.csv', ';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## Data analysis

分析一下有什么啊

| col | method | meaning | more |
| ----- | ------------- | ----------- | --- |
| age：| 原始数据 | 年龄 | numeric |
| job：| one-hot | 工作 | 字符串 |
| marital：| one-hot | 婚姻状态 | {"married","divorced","single"} 三种 |
| education：| one-hot | 教育水平 | {"unknown","secondary","primary","tertiary"} 四种 |
| default：| 0-1编码 | 有无违约信用 | yes/no |
| balance：| 原始数据 | 欧元，年平均余额 | numeric |
| housing：| 0-1 | 有无住房贷款 | yes/no |
| loan：| 0-1 | 有无私人贷款 | yes/no |
| contact：| one-hot | 联系方式 | {"unknown","telephone","cellular"} 三种 |
| day：| 原始数据 | 本月上次联络日 | numeric |
| month：| 整数编码 | 本年上次联络月 | {"jan", "feb", "mar", ..., "nov", "dec"} |
| duration：| 原始数据 | 上次联络时长 | numeric |
| campaign：| 原始数据 | number of contacts performed during this campaign and for this client | numeric |
| pdays：| 原始数据 | 上次和客户联络后经过的时间 | numeric |
| previous：| 原始数据 | 此次campaign之前和这个客户联络的次数 | numeric |
| poutcome：| one-hot | 此次之前的campaign的结果 | {"unknown","other","failure","success"} |
| y：| 0-1 | 客户是否订阅短期存款 | yes/no |

In [68]:
df["month"] = df["month"].astype(pd.CategoricalDtype(['jan', 'feb', 'mar','apr', 'may', 'jun', 'jul', 'aug', 'sep',  'oct', 'nov', 'dec'], ordered=True)).cat.codes
dummies_df = pd.get_dummies(df, columns=["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome", "y"])

In [65]:
ddf.columns

Index(['age', 'balance', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'poutcome_failure',
       'poutcome_other', 'poutcome_success', 'poutcome_unknown', 'y_no',
       'y_yes'],
      dtype='object')

In [69]:
dummies_df.head()

,age,balance,day,month,duration,campaign,pdays,previous,job_admin.,job_blue-collar,...,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
0,30,1787,19,9,79,1,-1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,33,4789,11,4,220,1,339,4,0,0,...,1,1,0,0,1,0,0,0,1,0
2,35,1350,16,3,185,1,330,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,30,1476,3,5,199,4,-1,0,0,0,...,1,0,0,1,0,0,0,1,1,0
4,59,0,5,4,226,1,-1,0,0,1,...,0,0,0,1,0,0,0,1,1,0
